In [135]:
import pandas as pd
import sqlite3
from prophet import Prophet
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").disabled=True

In [131]:
conn = sqlite3.connect('./data/trader_data.db')

query = """
            SELECT 
                ticker
                ,DATE(Date) AS Date
                ,Close
                ,GICSSector
                ,GICSSectorSub
                ,CompanyName
            FROM equity
            WHERE (1=1)
            AND Date > '1-1-2024'
            AND Close NOT NULL
            """

df = pd.read_sql_query(query, conn)

In [141]:
ticker_list = list(df['Ticker'].unique())

predictions = pd.DataFrame()

total = len(ticker_list)

for num, ticker in enumerate(ticker_list):
    
    progress = round((num / total) * 100, 2)
    
    print(str(progress) + ' %')
    
    stock_data = df[df['Ticker'] == ticker][['Date', 'Close']].reset_index(drop = True)
    
    stock_data.columns = ['ds', 'y']
    
    m = Prophet()
    
    m.fit(stock_data)
    
    future = m.make_future_dataframe(periods = 14)
    
    forecast = m.predict(future)
    
    forecast = forecast[['ds', 'yhat']].tail(14)
    
    forecast['Ticker'] = ticker
    
    predictions = pd.concat([forecast, predictions])
    
predictions.columns = ['Date', 'Close', 'Ticker']
    
predictions['Prediction'] = 1

predictions['Volume'] = 0

predictions['Open'] = 0 

predictions['Low'] = 0

predictions['High'] = 0

predictions['AdjClose'] = 0

0.0 %
0.2 %
0.4 %
0.6 %
0.8 %
0.99 %
1.19 %
1.39 %
1.59 %
1.79 %
1.99 %
2.19 %
2.39 %
2.58 %
2.78 %
2.98 %
3.18 %
3.38 %
3.58 %
3.78 %
3.98 %
4.17 %
4.37 %
4.57 %
4.77 %
4.97 %
5.17 %
5.37 %
5.57 %
5.77 %
5.96 %
6.16 %
6.36 %
6.56 %
6.76 %
6.96 %
7.16 %
7.36 %
7.55 %
7.75 %
7.95 %
8.15 %
8.35 %
8.55 %
8.75 %
8.95 %
9.15 %
9.34 %
9.54 %
9.74 %
9.94 %
10.14 %
10.34 %
10.54 %
10.74 %
10.93 %
11.13 %
11.33 %
11.53 %
11.73 %
11.93 %
12.13 %
12.33 %
12.52 %
12.72 %
12.92 %
13.12 %
13.32 %
13.52 %
13.72 %
13.92 %
14.12 %
14.31 %
14.51 %
14.71 %
14.91 %
15.11 %
15.31 %
15.51 %
15.71 %
15.9 %
16.1 %
16.3 %
16.5 %
16.7 %
16.9 %
17.1 %
17.3 %
17.5 %
17.69 %
17.89 %
18.09 %
18.29 %
18.49 %
18.69 %
18.89 %
19.09 %
19.28 %
19.48 %
19.68 %
19.88 %
20.08 %
20.28 %
20.48 %
20.68 %
20.87 %
21.07 %
21.27 %
21.47 %
21.67 %
21.87 %
22.07 %
22.27 %
22.47 %
22.66 %
22.86 %
23.06 %
23.26 %
23.46 %
23.66 %
23.86 %
24.06 %
24.25 %
24.45 %
24.65 %
24.85 %
25.05 %
25.25 %
25.45 %
25.65 %
25.84 %
26.04 %
26.24 %
2

In [142]:
sector = df[['Ticker','GICSSector', 'GICSSectorSub', 'CompanyName']].drop_duplicates()

final_df = sector.merge(predictions[['Prediction', 'Volume', 'Open', 'Low', 'High', 'AdjClose', 'Ticker', 'Close','Date']], on='Ticker' )

In [144]:
final_df.to_sql('equity', conn, if_exists = 'append', index = False)

7042

In [149]:
conn = sqlite3.connect('./data/trader_data.db')

query = """SELECT 
                 ticker
                ,ROUND(close, 2) as Close
                ,DATE(Date) AS Date
                ,GICSSector
                ,GICSSectorSub
                ,CompanyName
                ,CASE WHEN Prediction = 1 THEN 'ON' ELSE 'OFF' END AS Prediction
            FROM equity 
            WHERE (1=1)
                AND Date > '2024-01-01'
                AND close IS NOT NULL"""

tableau_data = pd.read_sql_query(query, conn)

tableau_data.to_excel('2024_stock_data.xlsx')